In [17]:
import pandas as pd
import sklearn
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score


import categorical_embedder as ce
import numpy as np

In [18]:
df = pd.read_csv('../Machine_Learning/df2.csv')

In [19]:
df

,y_true,symptom,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,...,DE,De,dE,de,SERIOUS Y,SERIOUS N,manufacturer_JANSSEN,manufacturer_MODERNA,manufacturer_NOVAVAX,manufacturer_PFIZER\BIONTECH
0,1,Angioedema,23,95,161,304,308,287,191,105,...,2878.0,943022.0,2351.0,1530710.0,564.0,2314.0,0,0,0,1
1,1,Hypoaesthesia,39,732,2295,4248,4426,3566,1960,862,...,26529.0,919371.0,27845.0,1505216.0,7405.0,19126.0,0,0,0,1
2,1,Erythema multiforme,20,12,31,28,41,30,34,30,...,733.0,945167.0,2228.0,1530833.0,131.0,602.0,0,0,0,1
3,1,Insomnia,33,205,732,1345,1345,1253,1012,581,...,11215.0,934685.0,14609.0,1518452.0,2674.0,8541.0,0,0,0,1
4,1,Myalgia,106,935,4266,7133,6992,6599,4653,2335,...,52182.0,893718.0,75910.0,1457151.0,12832.0,39351.0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1120,0,Increased insulin requirement,0,0,1,1,0,0,1,0,...,3.0,98389.0,33.0,2380536.0,2.0,1.0,1,0,0,0
1121,0,Blood lactate dehydrogenase decreased,0,0,0,0,1,0,0,0,...,7.0,945893.0,17.0,1533044.0,4.0,3.0,0,0,0,1
1122,0,Breast cellulitis,0,0,0,0,2,0,0,2,...,4.0,537275.0,16.0,1941666.0,1.0,3.0,0,1,0,0
1123,0,Vascular graft thrombosis,0,0,0,0,0,2,0,1,...,3.0,537276.0,10.0,1941672.0,3.0,0.0,0,1,0,0


In [20]:
X = df[['symptom']]
y = df['y_true']
print(X.shape)
print(y.shape)

(1125, 1)
(1125,)


In [21]:
y.values

array([1, 1, 1, ..., 0, 0, 0])

In [22]:
embedding_info = ce.get_embedding_info(X)
X_encoded, encoders = ce.get_label_encoded_data(X)

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y)

# Get embeddings
embeddings = ce.get_embeddings(X_train, y_train, categorical_embedding_info=embedding_info,
                               is_classification=True, epochs=100, batch_size=256)


Epoch 1/100
3/3 [==============================] - 1s 62ms/step - loss: 0.6552 - accuracy: 0.8887 - val_loss: 0.5828 - val_accuracy: 0.8817
Epoch 2/100
3/3 [==============================] - 0s 20ms/step - loss: 0.5401 - accuracy: 0.8887 - val_loss: 0.4682 - val_accuracy: 0.8817
Epoch 3/100
3/3 [==============================] - 0s 15ms/step - loss: 0.4156 - accuracy: 0.8887 - val_loss: 0.3743 - val_accuracy: 0.8817
Epoch 4/100
3/3 [==============================] - 0s 18ms/step - loss: 0.3285 - accuracy: 0.8887 - val_loss: 0.3498 - val_accuracy: 0.8817
Epoch 5/100
3/3 [==============================] - 0s 20ms/step - loss: 0.3053 - accuracy: 0.8887 - val_loss: 0.3600 - val_accuracy: 0.8817
Epoch 6/100
3/3 [==============================] - 0s 17ms/step - loss: 0.2885 - accuracy: 0.8887 - val_loss: 0.3436 - val_accuracy: 0.8817
Epoch 7/100
3/3 [==============================] - 0s 18ms/step - loss: 0.2438 - accuracy: 0.8887 - val_loss: 0.3136 - val_accuracy: 0.8817
Epoch 8/100
3/3 [===

In [23]:
df_embedded = ce.fit_transform(df, embeddings=embeddings, encoders=encoders, drop_categorical_vars=True)
df_embedded.head()

,y_true,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,...,symptom_embedding_40,symptom_embedding_41,symptom_embedding_42,symptom_embedding_43,symptom_embedding_44,symptom_embedding_45,symptom_embedding_46,symptom_embedding_47,symptom_embedding_48,symptom_embedding_49
0,1,23,95,161,304,308,287,191,105,45,...,0.027804,0.023666,0.003403,-0.010580,-0.038636,-0.033641,-0.014515,-0.044567,0.004077,0.024810
1,1,39,732,2295,4248,4426,3566,1960,862,241,...,-0.053546,-0.071147,-0.063355,-0.023203,0.005357,-0.005699,-0.055387,0.047468,0.067334,0.048406
2,1,20,12,31,28,41,30,34,30,20,...,-0.021373,-0.042889,-0.036618,0.022410,0.019555,-0.025536,-0.027378,-0.012780,0.044856,0.032292
3,1,33,205,732,1345,1345,1253,1012,581,180,...,0.017694,-0.016068,0.042424,-0.024262,-0.010284,-0.035142,0.038593,0.047516,-0.000315,-0.035767
4,1,106,935,4266,7133,6992,6599,4653,2335,583,...,-0.033978,-0.075673,-0.051500,-0.045333,0.097312,0.039915,-0.019549,0.042446,0.066608,0.039786


In [24]:
num_attribs = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '90-99', '100-109',
               '110-119', 'NUMDAYS_0-1', 'NUMDAYS_2-3', 'NUMDAYS_4-5', 'NUMDAYS_6-7', 'NUMDAYS_8-9', 'NUMDAYS_10-11',
               'NUMDAYS_12-13', 'NUMDAYS_14+', 'DIED', 'L_THREAT', 'HOSPITAL', 'X_STAY', 'DISABLE', 'BIRTH_DEFECT',
               'DE', 'De', 'dE', 'de', 'SERIOUS Y', 'SERIOUS N', 'SEX_M', 'SEX_F', 'SEX_U']

# Separate features and target
X = df_embedded.drop(columns=['y_true'])
y_true = df_embedded['y_true']

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('min_max_scaler', MinMaxScaler())
])

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs)
], remainder='passthrough')

# Apply transformation
X_transformed = full_pipeline.fit_transform(X)

# Combine the transformed features with the target
df_transformed = pd.DataFrame(X_transformed, columns=X.columns)

In [38]:
df_transformed.columnsc

Index(['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
       '80-89', '90-99', '100-109', '110-119', 'NUMDAYS_0-1', 'NUMDAYS_2-3',
       'NUMDAYS_4-5', 'NUMDAYS_6-7', 'NUMDAYS_8-9', 'NUMDAYS_10-11',
       'NUMDAYS_12-13', 'NUMDAYS_14+', 'SEX_M', 'SEX_F', 'SEX_U', 'DIED',
       'L_THREAT', 'HOSPITAL', 'X_STAY', 'DISABLE', 'BIRTH_DEFECT', 'DE', 'De',
       'dE', 'de', 'SERIOUS Y', 'SERIOUS N', 'manufacturer_JANSSEN',
       'manufacturer_MODERNA', 'manufacturer_NOVAVAX',
       'manufacturer_PFIZER\BIONTECH', 'symptom_embedding_0',
       'symptom_embedding_1', 'symptom_embedding_2', 'symptom_embedding_3',
       'symptom_embedding_4', 'symptom_embedding_5', 'symptom_embedding_6',
       'symptom_embedding_7', 'symptom_embedding_8', 'symptom_embedding_9',
       'symptom_embedding_10', 'symptom_embedding_11', 'symptom_embedding_12',
       'symptom_embedding_13', 'symptom_embedding_14', 'symptom_embedding_15',
       'symptom_embedding_16', 'symptom_embedding_1

In [36]:
X_train, X_test, y_train, y_test = train_test_split(df_transformed, y_true, test_size=0.2, random_state=42)

X_train.to_csv('../Machine_Learning/X_train.csv', index=False)
X_test.to_csv('../Machine_Learning/X_test.csv', index=False)
y_train.to_csv('../Machine_Learning/y_train.csv', index=False)
y_test.to_csv('../Machine_Learning/y_test.csv', index=False)